<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/RAG_lesson_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RAG pipeline cơ bản (với LlamaIndex và Llama 3)**

Đây là notebook cho các bạn để có thể tìm hiểu về cách tạo ra một mô hình RAG đơn giản, với LlamaIndex và open-source LLM như Llama 3.

Lời cảm ơn đặc biệt dành cho Plaban Nayak: [Build an Advanced Reranking-RAG System Using Llama-Index, Llama 3 and Qdrant](https://medium.com/@nayakpplaban/build-an-advanced-reranking-rag-system-using-llama-index-llama-3-and-qdrant-a8b8654174bc), vì ý tưởng cho notebook này.

***Cài đặt các thư viện cần thiết:***

Trước tiên, việc cần làm đầu tiên là cài đặt các thư viện cần thiết cho RAG pipeline. Một số điều cần lưu ý:
- LlamaIndex là framework chính chúng ta sẽ sử dụng, nó có hỗ trợ cho người dùng với kết nối LLM và các mô hình cho embeddings.
- Embeddings và LLM ở đây đều là mô hình mã nguồn mở, nên các mô hình này sẽ cần HuggingFace tokens (chìa khóa để kết nối với HuggingFace).
- sentence-transformers là phiên bản riêng để kích hoạt thư viện transformers cho xử lý câu trong NLP.

In [1]:
%%writefile requirements.txt
llama-index
llama-index-llms-huggingface
llama-index-embeddings-fastembed
bitsandbytes
llama-index-embeddings-huggingface
fastembed
deepeval
einops
accelerate
sentence-transformers

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

***Nhập tài liệu (Loading):***

Tiếp theo đó, các bạn cần nhập tài liệu vào trong vector database.

Ở notebook này, tài liệu chúng ta lựa chọn là một trích đoạn trong bản dịch tiếng Việt của "Trận chiến cuối cùng", một tác phẩm về trận Berlin trong thế chiến II: [Wikipedia](https://en.wikipedia.org/wiki/The_Last_Battle_(Ryan_book)).

Tài liệu sẽ được đưa vào VectorStoreIndex (vector database cơ bản nhất của LlamaIndex), để lưu trữ cho query của các bạn.

In [4]:
!wget "https://raw.githubusercontent.com/hoangcuongnguyen2001/RAG_lessons/main/Last_battle.txt" "Last_battle.txt"

--2024-07-01 06:23:05--  https://raw.githubusercontent.com/hoangcuongnguyen2001/RAG_lessons/main/Last_battle.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11281 (11K) [text/plain]
Saving to: ‘Last_battle.txt.1’

Last_battle.txt.1   100%[===================>]  11.02K  --.-KB/s    in 0s      

2024-07-01 06:23:06 (123 MB/s) - ‘Last_battle.txt.1’ saved [11281/11281]

--2024-07-01 06:23:06--  http://last_battle.txt/
Resolving last_battle.txt (last_battle.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘last_battle.txt’
FINISHED --2024-07-01 06:23:06--
Total wall clock time: 0.2s
Downloaded: 1 files, 11K in 0s (123 MB/s)


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["Last_battle.txt"]
).load_data()

Tiếp theo đó, các bạn có thể cài đặt system prompt (prompt hướng dẫn cho LLM nên làm gì), cùng với query wrapper prompt (để quy định cấu trúc cho query của người hỏi trước khi nhập vào LLM). Lưu ý, query wrapper prompt của mỗi LLM có thể tìm được trên model card tại HuggingFace; và trong ví dụ này mình sử dụng wrapper prompt từ Llama-3-8B-Instruct model card. Argument này có thể không cần thiết với các model khác.

Một điều đáng lưu ý nữa là nếu không có system prompt, hành vi ngầm định khi hỏi Llama3 bằng tiếng Việt là nó có thể thêm tiếng Anh trong câu trả lời.

In [6]:

from llama_index.core import PromptTemplate
system_prompt = """Bạn là một trợ lý AI đắc lực. Hãy trả lời các câu hỏi càng chính xác càng tốt.
 Lưu ý là bạn chỉ trả lời bằng tiếng Việt và không dùng ngôn ngữ nào khác. Hãy in ra kết quả dưới dạng JSON,
 với <result></result> tags ở đầu và cuối câu trả lời của bạn.
"""
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

***Indexing và lưu trữ tài liệu:***

Sau khi nhập tài liệu vào database, việc tiếp theo sẽ là tạo embedding cho từng phần nhỏ một (chunk), trước khi nhập embedding vào vector store index (đã được kích hoạt trước đó).

Lưu ý: embedding model chúng ta cần dùng nên là một mô hình hỗ trợ embedding cho nhiều ngôn ngữ (ở đây là *multilingual-e5-base*), hoặc một mô hình được huấn luyện với tài liệu của ngôn ngữ chúng ta muốn dùng trong câu hỏi. Với công nghệ hiện tại, chúng ta chưa thể chọn một mô hình embedding bất kỳ cho một LLM bất kỳ được.

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base", trust_remote_code=True)

Tiếp theo đó, tokenizer và LLM sẽ được tải lên cùng nhau (ở đây cả tokenizer và LLM đều từ chung một mô hình).

Stoppings_ids ở đây được sử dụng để giúp cho một LLM có thể hiểu được về việc kết thúc của từng câu là gì (nó tùy thuộc vào cấu trúc của từng LLM, như trong notebook này thì stopping_ids có thể tìm trong model card).

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


*Lưu ý khi nhập và setup LLM:*

1. Do môi trường làm việc của chúng ta là bản free của Google Colab (chỉ có 15GB RAM), các bạn nên sử dụng quantization (4 hoặc 8 bit) tùy thuộc vào kích cỡ LLM, để có thể nhập được model vào server.

2. Các bạn có thể setup temperature trong ngưỡng từ 0 tới 1, nên để temperature ở mức gần 0 nhất có thể để đảm bảo cho kết quả được chính xác.

3. Context windowm, chunk size và max new tokens ở đây đều là cài đặt ngầm định của Llama3-8B-Instruct.

In [17]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=8192,
    max_new_tokens=256,
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.1,
        "top_p": 0.3,

    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
    response_format= "json",
)

SyntaxError: positional argument follows keyword argument (<ipython-input-17-d851b2c7f204>, line 34)

In [10]:
from llama_index.core import Settings


Settings.embed_model = embed_model

Settings.chunk_size = 512
# Llama-3-8B-Instruct model
Settings.llm = llm

Tiếp theo, embedding sẽ được lưu trữ vào VectorStoreIndex (với LlamaIndex).

In [11]:
index = VectorStoreIndex.from_documents(documents)

***Đặt câu hỏi cho LLM (querying):***

Bước cuối cùng là đặt câu hỏi cho LLM thông qua vector database (như trong ví dụ này, chúng ta có thể đặt câu hỏi thông qua VectorStoreIndex (as_query_engine).

Lưu ý: Chúng ta có thể điều chỉnh similarity_top_k (top bao nhiêu từ mà LLM đoán là có khả năng nối tiếp từ cũ). Thông thường, similarity_top_k càng thấp thì kết quả đưa ra càng chính xác.

In [12]:
query_engine = index.as_query_engine(similarity_top_k=3, streaming=True)
#query_engine = index.as_query_engine(similarity_top_k=2)

In [13]:
response = query_engine.query("Chuyện gì đã xảy ra với Sở thú Berlin?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Câu trả lời có thể được in trực tiếp (dùng *print()*), hoặc in theo từng từ một, giống như cách ChatGPT hoặc các LLM khác đưa ra câu trả lời (dùng *print_response_stream()*).

In [14]:
#response.print_response_stream()
print(response)

Sở thú Berlin đã bị hư hại nghiêm trọng, giờ chỉ còn là một khu rừng đổ nát. Hồ cá đã bị hủy hoại hoàn toàn. Các khu nuôi bò sát, hà mã, kangaroo, hổ và voi cũng bị hư hại nghiêm trọng, cùng với các tòa nhà bị thủng toang hoác khác.


In [15]:
from transformers import AutoModelForCausalLM
from deepeval.models.base_model import DeepEvalBaseLLM

class Llama3(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Llama 3"

model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True,
                                              quantization_config=quantization_config
    )
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

llama3 = Llama3(model=model, tokenizer=tokenizer)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
print(llama3.generate("What have happened in Berlin Zoo in 1945?"))



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>What have happened in Berlin Zoo in 1945? - History
Berlin Zoo, which is one of the oldest and most famous zoos in the world, has a dark and tragic history during World War II. In 1945, the zoo was severely damaged and many animals died due to the war and the subsequent Soviet occupation.
In the final months of World War II, Berlin was heavily bombed by the Allies, and the zoo was not spared. On February 3, 1945, a British air raid damaged the zoo's buildings and killed many animals. The zoo's director, Lutz Heck, was away at the time, and his wife, Margarete, was left in charge of the zoo.
As the Soviet army approached Berlin, the zoo's staff and animals were in danger. Margarete Heck and her team worked tirelessly to care for the remaining animals, but it was a difficult and desperate situation. Many animals had died due to the bombing, and the zoo's resources were dwindling.
On April 20, 1945, the Soviet army occupied Berlin, and the zoo was taken over by the Red Ar

In [24]:
from deepeval import evaluate
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

actual_output = ["""
Sở thú Berlin đã bị hư hại nghiêm trọng. Các khu nuôi bò sát, hà mã, kangaroo, hổ và voi đã bị hư hại
nghiêm trọng, cùng với các tòa nhà bị thủng toang hoác khác. Hồ cá đã bị hủy hoại hoàn toàn.

"""]


retrieval_context = ["""
Cách đó gần 100m là Sở thú Berlin danh tiếng thế giới, giờ chỉ còn là một khu rừng đổ nát.
Hồ cá đã bị hủy hoại hoàn toàn. Các khu nuôi bò sát, hà mã, kangaroo, hổ và voi cũng bị hư hại nghiêm trọng,
 cùng với các tòa nhà bị thủng toang hoác khác. Công viên Tiergarten nổi tiếng rộng 630 mau Anh nằm xung quanh
  đó giờ thành vùng đất không người, chỉ còn trơ lại những hố bom to bằng cả căn phòng, những hồ nước đầy gạch
  vụn và tòa đại sứ đã hư hại một phần. Công viên từng là một khu rừng thiên nhiên đầy kỳ hoa di thảo. Giờ
   những cái cây quý giá ấy đã bị thiêu trụi, trơ lại mây gốc cây xấu xí.
"""]
metric = ContextualRelevancyMetric(
    threshold=0.7,
    model=llama3,
    include_reason=True
)
test_case = LLMTestCase(
    input="Chuyện gì đã xảy ra với Sở thú Berlin?",
    actual_output=actual_output,
    retrieval_context=retrieval_context
)

print(metric)
metric.measure(test_case)
print(metric.score)
print(metric.reason)

# # or evaluate test cases in bulk
# evaluate([test_case], [metric])

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ValueError: Evaluation LLM outputted an invalid JSON. Please use a better evaluation model.